# Inaugural Project

Slaget i Køge Bugt

In [1]:
import numpy as np

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import inauguralproject

# Question 1

First, we define the parameters that enter the utility function and the resource constraint.

In [2]:
money=1 #m
v=10 #v
frisch=0.3 #epsilon
ltax=0.4 #tau_0
ttax=0.1 #tau_1
cutoff=0.4 #kappa
w=0.5

Now, define the labour supply utility function and the budget constraint function.

In [3]:
def laboursupply(l,c,frisch,v):
    #Returning utility from labour supply and consumption
    u=np.log(c)-v*l**(1+1/frisch)/(1+1/frisch)
    return (u)

def budget_constraint(money,w,l,ltax,ttax,cutoff):
    #Returning total resources in terms of wage less taxes and cash
    budget=money+w*l-w*l*ltax-ttax*max((w*l-cutoff),0)
    return budget


As we have now defined the functions that we wish to maximise, we move to solving the labour supply problem. Note that we apply:
$$c^\star=x$$
i.e. that as utility is monotonically increasing in consumption, the consumer will always spend his entire budget $x$ on consumption $c$. We solve the agent's labour supply problem using the $optimize$ command imported from $scipy$. For a given labour supply, we return the possible level of consumption and the negative value of utility following from these pairs of $(c,l)$. We then minimise this negative utility to find the optimal $(c^\star,l^\star)$.

In [5]:
from scipy import optimize

#Objective function returning negative utility, to minimize
def value_of_choice_budget(l,w,frisch,v,ltax,ttax,cutoff):
    c=budget_constraint(money,w,l,ltax,ttax,cutoff)
    return -laboursupply(l,c,frisch,v)

def optimiser(w,frisch,v,ltax,ttax,cutoff,money):
    #Call the solver
    sol_case1 = optimize.minimize_scalar(
        value_of_choice_budget,method='bounded',
        bounds=(0,1),args=(w,frisch,v,ltax,ttax,cutoff))

    #Return optimal labour supply and consumption
    lstar = sol_case1.x
    cstar = budget_constraint(money,w,lstar,ltax,ttax,cutoff)
    ustar = laboursupply(lstar,cstar,frisch,v)
    return [lstar,cstar,ustar]


# Question 2

ADD ANSWER.

In [6]:
import matplotlib.pyplot as plt
plt.style.use("seaborn")

N=1000
w_vec=np.linspace(0.5,1.5,N)
l_vec=np.empty(N)
c_vec=np.empty(N)
for i,w in enumerate(w_vec):
    bundle=optimiser(w,frisch,v,ltax,ttax,cutoff,money)
    l_vec[i]=bundle[0]
    c_vec[i]=bundle[1]

# a. create a figure
fig = plt.figure(figsize=(10,4))

# c. left plot
ax_left = fig.add_subplot(1,2,1)

ax_left.plot(w_vec,l_vec)

ax_left.set_title('Optimal labour supply given wage')
ax_left.set_xlabel('$w$')
ax_left.set_ylabel('$l^\star$')
ax_left.grid(True)

# c. right plot
ax_right = fig.add_subplot(1,2,2)

ax_right.plot(w_vec,c_vec)

ax_right.set_title('Optimal consumption given wage')
ax_right.set_xlabel('$w$')
ax_right.set_ylabel('$c^\star$')
ax_right.grid(True)

# Question 3

We first draw a vector of individual wages $w_i$ where $w_i\sim U((0,5),(1,5))$. 

In [7]:
np.random.seed(4600)
PopWage = np.random.uniform(0.5,1.5,size=10000)
    
def TotalTax(PopWages,frisch,v,ltax,ttax,cutoff,money):
    N=len(PopWage)
    PopTaxes=np.zeros((N))

    for i,w in enumerate(PopWage):
        Ind_optimum=optimiser(w,frisch,v,ltax,ttax,cutoff,money)
        IndLabour=Ind_optimum[0]
        PopTaxes[i]=ltax*w*IndLabour+ttax*max(w*IndLabour-cutoff,0)
    TotTax=sum(PopTaxes)
    return TotTax

TotalTax(PopWage,frisch,v,ltax,ttax,cutoff,money)

1627.1515816296296

# Question 4

ADD ANSWER.

In [8]:
InelasticFrisch = 0.1
TotalTax(PopWage,InelasticFrisch,v,ltax,ttax,cutoff,money)

3191.15863958847

# Question 5

Mulig løsning, der finder optimal kollektiv skatterate.

In [15]:
PopWage = np.random.uniform(0.5,1.5,size=1000)
def value_of_choice_tax(taxes,PopWage,frisch,v,money):
    
    ltax = taxes[0]
    ttax = taxes[1]
    cutoff = taxes[2]
    return -TotalTax(PopWage,frisch,v,ltax,ttax,cutoff,money)

# b. call solver
initial_guess = [0.785,0.055,0.470]
sol_case3 = optimize.minimize(
    value_of_choice_tax,initial_guess,method='Nelder-Mead',
    args=(PopWage,frisch,v,money))

ltaxStar=sol_case3.x[0]
ttaxStar=sol_case3.x[1]
cutoffStar=sol_case3.x[2]

print(f'Optimal lower tax rate is {ltaxStar:.3f}')
print(f'Optimal upper tax rate is {ttaxStar:.3f}')
print(f'Optimal cutoff income is {cutoffStar:.3f}')

Optimal lower tax rate is 0.786
Optimal upper tax rate is 0.054
Optimal cutoff income is 0.531


# Conclusion

ADD CONCISE CONLUSION.